# 第5章: 発展的なグラフ技術

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### ライブラリのインストール

このセルは、LangGraphおよび関連するLangChainライブラリをインストールします。実行には数分かかる場合があります。
ご利用になるLLMプロバイダーに応じて、コメントアウトを解除して必要なライブラリをインストールしてください。

In [ ]:
# === ライブラリのインストール ===
# 基本ライブラリ (LangGraphとLangChain Core)
!%pip install -qU langchain langgraph

# --- LLMプロバイダー別ライブラリ ---
# ご利用になるLLMプロバイダーに応じて、以下の該当する行のコメントを解除して実行してください。

# OpenAI (GPTシリーズ)
# !%pip install -qU langchain_openai

# Azure OpenAI
# !%pip install -qU langchain_openai # Azureもlangchain_openaiを利用

# Google Cloud Vertex AI (Gemini, PaLM等)
# !%pip install -qU langchain_google_vertexai

# Google Gemini API (Google AI Studioで利用するGemini)
# !%pip install -qU langchain_google_genai

# Anthropic (Claudeシリーズ)
# !%pip install -qU langchain_anthropic

# Amazon Bedrock (AWS上の各種モデル、Claudeも含む)
# !%pip install -qU langchain_aws boto3 # Bedrock利用時はboto3も必要

# --- その他の推奨ライブラリ ---
# グラフの可視化や環境変数管理など、演習全体を通して利用する可能性のあるライブラリ
!%pip install -qU python-dotenv pygraphviz pydotplus graphviz

### LLMプロバイダーの選択

このセルでは、使用するLLMプロバイダーを選択します。
`LLM_PROVIDER` 変数に、利用したいプロバイダー名を設定してください。
選択可能なプロバイダー: `"openai"`, `"azure"`, `"google"` (Vertex AI), `"google_genai"` (Gemini API), `"anthropic"`, `"bedrock"`

In [ ]:
# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google" (Vertex AI), "google_genai" (Gemini API), "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合

### APIキー/環境変数の設定

以下のセルを実行する前に、選択したLLMプロバイダーに応じたAPIキーまたは環境変数を設定する必要があります。

**一般的な手順:**
1.  `.env.sample` ファイルをコピーして `.env` ファイルを作成します。
2.  `.env` ファイルを開き、選択したLLMプロバイダーに対応するAPIキーや必要な情報を記述します。
    *   **OpenAI:** `OPENAI_API_KEY`
    *   **Azure OpenAI:** `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, `OPENAI_API_VERSION`, `AZURE_OPENAI_DEPLOYMENT_NAME`
    *   **Google (Vertex AI):** `GOOGLE_CLOUD_PROJECT_ID`, `GOOGLE_CLOUD_LOCATION` (Colab環境外で実行する場合、`GOOGLE_APPLICATION_CREDENTIALS` 環境変数の設定も必要になることがあります)
    *   **Google (Gemini API):** `GOOGLE_API_KEY`
    *   **Anthropic:** `ANTHROPIC_API_KEY`
    *   **AWS Bedrock:** `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION_NAME` (IAMロールを使用する場合は、これらのキー設定は不要な場合がありますが、リージョン名は必須です)
3.  ファイルを保存します。

**Google Colab を使用している場合:**
上記の `.env` ファイルを使用する代わりに、Colabのシークレットマネージャーに必要なキーを登録してください。
例えば、OpenAIを使用する場合は `OPENAI_API_KEY` という名前でシークレットを登録します。
Vertex AI を利用する場合は、Colab上での認証 (`google.colab.auth.authenticate_user()`) が実行されます。

このセルは、設定された情報に基づいて環境変数をロードし、LLMクライアントを初期化します。

In [ ]:
# === APIキー/環境変数の設定 ===
import os
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む (存在する場合)
load_dotenv()

try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# --- OpenAI ---
if LLM_PROVIDER == "openai":
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY and IS_COLAB:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 OPENAI_API_KEY を設定するか、Colab環境の場合はシークレットに OPENAI_API_KEY を設定してください。")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Azure OpenAI ---
elif LLM_PROVIDER == "azure":
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
    OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if IS_COLAB:
        if not AZURE_OPENAI_API_KEY: AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
        if not AZURE_OPENAI_ENDPOINT: AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
        if not OPENAI_API_VERSION: OPENAI_API_VERSION = userdata.get("OPENAI_API_VERSION") # 例: "2023-07-01-preview"
        if not AZURE_OPENAI_DEPLOYMENT_NAME: AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not AZURE_OPENAI_API_KEY: raise ValueError("Azure OpenAI APIキー (AZURE_OPENAI_API_KEY) が設定されていません。")
    if not AZURE_OPENAI_ENDPOINT: raise ValueError("Azure OpenAI エンドポイント (AZURE_OPENAI_ENDPOINT) が設定されていません。")
    if not OPENAI_API_VERSION: OPENAI_API_VERSION = "2023-07-01-preview" # デフォルトを設定することも可能
    if not AZURE_OPENAI_DEPLOYMENT_NAME: raise ValueError("Azure OpenAI デプロイメント名 (AZURE_OPENAI_DEPLOYMENT_NAME) が設定されていません。")

    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
    os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION

# --- Google Cloud Vertex AI (Gemini) ---
elif LLM_PROVIDER == "google":
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT_ID") # .env 用に修正
    LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")

    if IS_COLAB:
        if not PROJECT_ID: PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID")
        if not LOCATION: LOCATION = userdata.get("GOOGLE_CLOUD_LOCATION") # 例: "us-central1"
        from google.colab import auth as google_auth
        google_auth.authenticate_user() # Vertex AI を使う場合は Colab での認証を推奨
    else: # Colab外の場合、.envから読み込んだ値で環境変数を設定
        if PROJECT_ID: os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID # Vertex AI SDKが参照する標準的な環境変数名
        if LOCATION: os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION

    if not PROJECT_ID: raise ValueError("Google Cloud Project ID が設定されていません。環境変数 GOOGLE_CLOUD_PROJECT_ID を設定するか、Colab環境の場合はシークレットに GOOGLE_CLOUD_PROJECT_ID を設定してください。")
    if not LOCATION: LOCATION = "us-central1" # デフォルトロケーション

# --- Google Gemini API (langchain-google-genai) ---
elif LLM_PROVIDER == "google_genai":
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY and IS_COLAB:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google APIキーが設定されていません。環境変数 GOOGLE_API_KEY を設定するか、Colab環境の場合はシークレットに GOOGLE_API_KEY を設定してください。")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# --- Anthropic (Claude) ---
elif LLM_PROVIDER == "anthropic":
    ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY and IS_COLAB:
        ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("Anthropic APIキーが設定されていません。環境変数 ANTHROPIC_API_KEY を設定するか、Colab環境の場合はシークレットに ANTHROPIC_API_KEY を設定してください。")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# --- Amazon Bedrock (Claude) ---
elif LLM_PROVIDER == "bedrock":
    AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
    AWS_REGION_NAME = os.environ.get("AWS_REGION_NAME")

    if IS_COLAB: 
        if not AWS_ACCESS_KEY_ID: AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
        if not AWS_SECRET_ACCESS_KEY: AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
        if not AWS_REGION_NAME: AWS_REGION_NAME = userdata.get("AWS_REGION_NAME")

    if not AWS_REGION_NAME:
         raise ValueError("AWSリージョン名 (AWS_REGION_NAME) が設定されていません。Bedrock利用にはリージョン指定が必要です。")

    # 環境変数に設定 (boto3がこれらを自動で読み込む)
    if AWS_ACCESS_KEY_ID: os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    if AWS_SECRET_ACCESS_KEY: os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    os.environ["AWS_DEFAULT_REGION"] = AWS_REGION_NAME # boto3が参照する標準的なリージョン環境変数名
    os.environ["AWS_REGION"] = AWS_REGION_NAME # いくつかのライブラリはこちらを参照することもある

print(f"APIキー/環境変数の設定完了 (プロバイダー: {LLM_PROVIDER})")

### LLMクライアントの初期化

このセルは、上で選択・設定したLLMプロバイダーに基づいて、対応するLLMクライアントを初期化します。

In [ ]:
# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    llm = AzureChatOpenAI(
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"), # 環境変数から取得
        openai_api_version=os.environ.get("OPENAI_API_VERSION"), # 環境変数から取得
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    # PROJECT_ID, LOCATION は前のセルで環境変数に設定済みか、Colabの場合は直接利用
    llm = ChatVertexAI(model_name="gemini-1.5-flash-001", temperature=0, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location=os.environ.get("GOOGLE_CLOUD_LOCATION"))
elif LLM_PROVIDER == "google_genai":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0, convert_system_message_to_human=True)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock # langchain_community.chat_models から langchain_aws に変更の可能性あり
    # AWS_REGION_NAME は前のセルで環境変数 AWS_DEFAULT_REGION に設定済み
    llm = ChatBedrock( # BedrockChat ではなく ChatBedrock が一般的
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        # region_name=os.environ.get("AWS_DEFAULT_REGION"), # 通常、boto3が環境変数から自動で読み込む
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'google_genai', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
if llm:
    print(f"LLM Client Type: {type(llm)}")
    # モデル名取得の試行を汎用的に
    model_attr = getattr(llm, 'model', None) or \ 
                 getattr(llm, 'model_name', None) or \ 
                 getattr(llm, 'model_id', None) or \ 
                 (hasattr(llm, 'llm') and getattr(llm.llm, 'model', None)) # 一部のLLMクライアントのネスト構造に対応
    if hasattr(llm, 'azure_deployment') and not model_attr: # Azure特有の属性
        model_attr = llm.azure_deployment
        
    if model_attr:
        print(f"LLM Model: {model_attr}")
    else:
        print("LLM Model: (Could not determine model name from client attributes)")


---

この章では、Agent Swarm、永続化(Checkpointer)、ストリーミング、カスタムエージェントなど、LangGraphのより高度な機能やテクニックについて学びます。

### ■ 問題001: グラフ実行のストリーミング出力の基本

`app.stream()` メソッドを使用すると、グラフの実行中に各ノードの入出力やイベントをリアルタイムで監視できます。これにより、グラフがどのように動作しているかを詳細に把握したり、ユーザーに逐次的なフィードバックを提供したりすることが可能になります。この問題では、簡単な2ステップのグラフを作成し、その実行過程をストリーミングで取得する方法を学びます。

In [ ]:
# 解答欄001
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
import time

# --- 状態定義 (State) ---
class StreamingState(TypedDict):
    messages: Annotated[list, add_messages]
    processed_text: str

# --- ノード定義 (Nodes) ---
def entry_node(state: StreamingState):
    user_message = state["messages"][-1].content
    print(f"入力ノード実行: '{user_message}' を受信")
    time.sleep(0.5) # 処理を模倣
    return {"processed_text": f"入力: {user_message}"}

def processing_node(state: StreamingState):
    current_text = state["processed_text"]
    print(f"処理ノード実行: '{current_text}' を大文字に変換")
    time.sleep(0.5) # 処理を模倣
    processed_text = current_text.upper()
    return {"processed_text": processed_text, "messages": [AIMessage(content=f"処理結果: {processed_text}")]}

# --- グラフ構築 (Graph) ---
workflow = ____(StreamingState) 

workflow.add_node("entry", entry_node)
workflow.add_node("processor", processing_node)

workflow.____("entry") 
workflow.____("entry", "processor") 
workflow.____("processor", ____) 

app = workflow.____() 

# --- グラフの実行とストリーミング表示 ---
inputs = {"messages": [HumanMessage(content="こんにちはストリーミング")]}
print("\n--- ストリーミング開始 ---")
for event in app.____(inputs, stream_mode="values"): 
    print(event)
print("--- ストリーミング終了 ---")

# 参考: invokeでの最終結果
final_result = app.invoke(inputs)
print(f"\n最終結果 (invoke): {final_result}")

<details><summary>解答001</summary>

```python
# 解答001
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
import time

# --- 状態定義 (State) ---
class StreamingState(TypedDict):
    messages: Annotated[list, add_messages]
    processed_text: str

# --- ノード定義 (Nodes) ---
def entry_node(state: StreamingState):
    user_message = state["messages"][-1].content
    print(f"入力ノード実行: '{user_message}' を受信")
    time.sleep(0.5) # 処理を模倣
    return {"processed_text": f"入力: {user_message}"}

def processing_node(state: StreamingState):
    current_text = state["processed_text"]
    print(f"処理ノード実行: '{current_text}' を大文字に変換")
    time.sleep(0.5) # 処理を模倣
    processed_text = current_text.upper()
    return {"processed_text": processed_text, "messages": [AIMessage(content=f"処理結果: {processed_text}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(StreamingState)

workflow.add_node("entry", entry_node)
workflow.add_node("processor", processing_node)

workflow.set_entry_point("entry")
workflow.add_edge("entry", "processor")
workflow.add_edge("processor", END)

app = workflow.compile()

# --- グラフの実行とストリーミング表示 ---
inputs = {"messages": [HumanMessage(content="こんにちはストリーミング")]}
print("\n--- ストリーミング開始 ---")
for event in app.stream(inputs, stream_mode="values"):
    print(event)
print("--- ストリーミング終了 ---")

# 参考: invokeでの最終結果
final_result = app.invoke(inputs)
print(f"\n最終結果 (invoke): {final_result}")
```
</details>

<details><summary>解説001</summary>

#### この問題のポイント
*   **学習内容:** `app.stream()`メソッドの基本的な使い方と、それによって得られるイベントの種類を理解します。`stream_mode="values"`を指定すると、各ステップでの状態全体が辞書形式でストリーミングされます。
*   **コード解説:**
    *   `StreamingState`には、メッセージ履歴の他に、処理中のテキストを保持する`processed_text`フィールドを定義しました。
    *   `entry_node`は入力メッセージを受け取り、`processed_text`を初期化します。
    *   `processing_node`は`processed_text`を大文字に変換し、最終的なメッセージを`messages`に追加します。
    *   `app.stream(inputs, stream_mode="values")`を呼び出すと、イテレータが返されます。このイテレータをループ処理することで、グラフの各実行ステップ（ノードの実行完了時など）の状態を取得できます。
    *   出力される`event`は、`{'node_name': state_after_node_execution}` のような形式の辞書です。例えば、`{'entry': {'messages': [...], 'processed_text': '入力: こんにちはストリーミング'}}` や `{'processor': {'messages': [...], 'processed_text': '入力: こんにちはストリーミング'}}` (processor実行前の状態)、そして `{'processor': {'messages': [...], 'processed_text': '入力: コンニチハストリーミング'}}` (processor実行後の状態) などが出力されます。
    *   `stream_mode`には他にも`"updates"`（変更差分のみ）や`"events"`（より詳細なイベント情報）があります。`"values"`は状態全体を見たい場合に便利です。
    *   `time.sleep(0.5)` は、ストリーミングが逐次的であることを視覚的に分かりやすくするためのものです。
---
</details>

### ■ 問題002: LLMからのストリーミング応答をリアルタイムで処理する

LLM（大規模言語モデル）をグラフに組み込む際、応答が完了するまで待つのではなく、生成されるテキストをトークンごと、またはチャンクごとにリアルタイムで受け取りたい場合があります。これは特に、ユーザーインターフェースに逐次的にテキストを表示するチャットボットなどで重要です。
この問題では、LLMノードからの出力をストリーミングで受け取り、部分的な応答を処理する方法を学びます。`stream_mode="updates"` を使用して、変更があった部分のみを取得します。

In [ ]:
# 解答欄002
from typing import TypedDict, Annotated, AsyncIterator
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from langchain_core.runnables import RunnableConfig
import asyncio

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class LLMStreamingState(TypedDict):
    messages: Annotated[list, add_messages]
    current_response_chunk: str # LLMからの最新の応答チャンク

# --- ノード定義 (Nodes) ---
async def streaming_llm_node(state: LLMStreamingState, config: RunnableConfig) -> AsyncIterator[LLMStreamingState]:
    print("\nstreaming_llm_node: LLM呼び出し開始")
    # `llm.astream` を使ってストリーミング応答を取得
    async for chunk in ____.____(state["messages"], config=config): 
        print(f"  受信チャンク: {chunk.content}", end="", flush=True)
        yield {____: chunk.content} 
    print("\nstreaming_llm_node: LLM呼び出し完了")
    yield {"messages": [AIMessage(content="(ストリーム完了)")]} 

# --- グラフ構築 (Graph) ---
workflow = ____(LLMStreamingState) 

workflow.____("llm_streamer", streaming_llm_node) 
workflow.____("llm_streamer") 
workflow.____("llm_streamer", ____) 

app = workflow.compile()

# --- グラフの非同期実行とストリーミング表示 ---
async def main():
    inputs = {"messages": [HumanMessage(content="LangGraphのストリーミングについて3つのポイントで教えてください。")]}
    print("--- 非同期ストリーミング開始 ---")
    full_response_content = ""
    async for update_event in app.____(inputs, stream_mode=____): 
        for node_name, state_update in update_event.items():
            if "current_response_chunk" in state_update:
                chunk_content = state_update["current_response_chunk"]
                full_response_content += chunk_content
            else:
                print(f"\nイベント({node_name}): {state_update}")

    print("\n--- 非同期ストリーミング終了 ---")
    print(f"\n最終的な組み立てられた応答:\n{full_response_content}")

    final_result = await app.ainvoke(inputs)
    print(f"\n最終結果 (ainvoke): {final_result}")

if __name__ == '__main__':
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio
            nest_asyncio.apply()
            loop.run_until_complete(main())
        else:
            loop.run_until_complete(main())
    except RuntimeError as e:
        if "cannot run new tasks" in str(e) or "already running" in str(e):
            import nest_asyncio
            nest_asyncio.apply()
            asyncio.run(main())
        else:
            raise

<details><summary>解答002</summary>

```python
# 解答002
from typing import TypedDict, Annotated, AsyncIterator
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from langchain_core.runnables import RunnableConfig
import asyncio

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class LLMStreamingState(TypedDict):
    messages: Annotated[list, add_messages]
    current_response_chunk: str # LLMからの最新の応答チャンク

# --- ノード定義 (Nodes) ---
async def streaming_llm_node(state: LLMStreamingState, config: RunnableConfig) -> AsyncIterator[LLMStreamingState]:
    print("\nstreaming_llm_node: LLM呼び出し開始")
    # `llm.astream` を使ってストリーミング応答を取得
    async for chunk in llm.astream(state["messages"], config=config):
        # chunk は AIMessageChunk オブジェクトであることが多い
        print(f"  受信チャンク: {chunk.content}", end="", flush=True)
        yield {"current_response_chunk": chunk.content} # 部分的な更新をyield
    print("\nstreaming_llm_node: LLM呼び出し完了")
    # ストリームの最後に最終的なメッセージを messages に追加する (オプション)
    # この例では current_response_chunk をつなぎ合わせる処理は省略し、各チャンクをそのまま流す
    # 完全なメッセージを messages に追加したい場合は、別途状態更新ロジックが必要
    yield {"messages": [AIMessage(content="(ストリーム完了)")]} # 完了を示すメッセージ (内容は任意)

# --- グラフ構築 (Graph) ---
workflow = StateGraph(LLMStreamingState)

workflow.add_node("llm_streamer", streaming_llm_node)
workflow.set_entry_point("llm_streamer")
workflow.add_edge("llm_streamer", END)

app = workflow.compile()

# --- グラフの非同期実行とストリーミング表示 ---
async def main():
    inputs = {"messages": [HumanMessage(content="LangGraphのストリーミングについて3つのポイントで教えてください。")]}
    print("--- 非同期ストリーミング開始 ---")
    full_response_content = ""
    # stream_mode="updates" を使うと、変更があった部分のみが辞書で返る
    async for update_event in app.astream(inputs, stream_mode="updates"):
        for node_name, state_update in update_event.items():
            if "current_response_chunk" in state_update:
                chunk_content = state_update["current_response_chunk"]
                # print(f"イベント({node_name}): {chunk_content}", end="", flush=True) # ノード内でprintしているのでここでは省略
                full_response_content += chunk_content
            else:
                # messages の更新など、他の状態変化もここで確認可能
                print(f"\nイベント({node_name}): {state_update}")

    print("\n--- 非同期ストリーミング終了 ---")
    print(f"\n最終的な組み立てられた応答:\n{full_response_content}")

    # 参考: ainvokeでの最終結果
    final_result = await app.ainvoke(inputs)
    print(f"\n最終結果 (ainvoke): {final_result}")

# Jupyter Notebookで非同期コードを実行するためにイベントループを取得して実行
if __name__ == '__main__':
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # ネストされたイベントループが許可されていない場合、新しいイベントループを作成
            # (Jupyter Notebookなど、既にイベントループが実行中の環境向け)
            print("既存のイベントループが実行中です。新しいループで実行します。")
            import nest_asyncio
            nest_asyncio.apply()
            loop.run_until_complete(main())
        else:
            loop.run_until_complete(main())
    except RuntimeError as e:
        if "cannot run new tasks" in str(e) or "already running" in str(e):
            # フォールバックとして nest_asyncio を使う
            print(f"RuntimeError: {e}. nest_asyncio を使用して再試行します。")
            import nest_asyncio
            nest_asyncio.apply()
            asyncio.run(main()) # nest_asyncio.apply() 後は asyncio.run でも動作するはず
        else:
            raise
```
</details>

<details><summary>解説002</summary>

#### この問題のポイント
*   **学習内容:**
    1.  **非同期ノード:** ノード関数を `async def` で定義し、戻り値のアノテーションに `AsyncIterator[StateType]` を使用することで、ノード自体がストリーミング出力（部分的な状態更新）を行えるようになります。
    2.  **LLMのストリーミング:** `llm.astream()` メソッド（または対応するストリーミングメソッド）を使用して、LLMからの応答をチャンクで非同期に受け取ります。
    3.  **部分的な状態更新:** ノード内で `yield {key: value}` を使うことで、そのキーに対応する状態のみを更新し、グラフの呼び出し元にストリーミングします。
    4.  **`app.astream()` と `stream_mode="updates"`:** コンパイルされたグラフを `app.astream()` で呼び出し、`stream_mode="updates"` を指定することで、各ステップで変更があった状態の部分だけを効率的に受け取ることができます。
    5.  **Jupyter Notebookでの非同期実行:** `asyncio.get_event_loop().run_until_complete()` や `nest_asyncio` を使って、Jupyter Notebookのような既にイベントループが実行されている可能性のある環境で非同期コードを実行する方法を学びます。
*   **コード解説:**
    *   `LLMStreamingState` に `current_response_chunk` を追加し、LLMからの最新のテキストチャンクを保持します。
    *   `streaming_llm_node`:
        *   `async def` で定義され、`AsyncIterator[LLMStreamingState]` を返します。
        *   内部で `llm.astream(state["messages"], config=config)` を呼び出します。`config` 引数は、LangChainの実行コンテキスト（コールバックなど）を伝播させるために重要です。
        *   `llm.astream` から得られた各 `chunk`（通常は `AIMessageChunk` オブジェクト）の内容を `yield {"current_response_chunk": chunk.content}` のようにしてストリーミングします。これにより、`current_response_chunk` のみが更新されたイベントがグラフの外部に送られます。
        *   最後に `yield {"messages": [AIMessage(content="(ストリーム完了)")]}` で `messages` 状態を更新しています。実際のアプリケーションでは、全てのチャンクを結合して完全なAIメッセージとして `messages` に追加することが一般的です。
    *   `app.astream(inputs, stream_mode="updates")`:
        *   非同期でグラフを実行し、ストリーミング出力を受け取ります。
        *   `stream_mode="updates"` のため、イベントは `{"node_name": {"state_key_updated": new_value}}` のような形式になります。この例では `{"llm_streamer": {"current_response_chunk": "some text"}}` や `{"llm_streamer": {"messages": [...]}}` といった更新が流れてきます。
    *   `main` 関数内のループでは、`update_event` から `current_response_chunk` を取り出し、`full_response_content` に結合しています。
    *   `if __name__ == '__main__':` ブロックは、Jupyter Notebook環境で `asyncio` のイベントループを適切に処理するための一般的なパターンです。`nest_asyncio` は、既に実行中のイベントループ内でさらに非同期コードを実行可能にするために使われます。
---
</details>

### ■ 問題003: `MemorySaver` を使ったグラフ状態の永続化と再開

LangGraphでは、グラフの実行状態を保存し、後で同じ状態から再開することができます。これを実現するのがチェックポイント機能です。`MemorySaver` は、状態をインメモリに保存する最もシンプルなチェックポインターです。
この問題では、`MemorySaver` を使ってグラフの実行状態を一時的に保存し、異なる `invoke` 呼び出し間で状態が引き継がれる（再開される）ことを確認します。スレッドID (`thread_id`) を使って、特定の実行シーケンスを識別します。

In [ ]:
# 解答欄003
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver # MemorySaverをインポート
import uuid

# --- 状態定義 (State) ---
class CheckpointState(TypedDict):
    messages: Annotated[list, add_messages]
    counter: int

# --- ノード定義 (Nodes) ---
def counter_node(state: CheckpointState):
    print(f"counter_node: 現在のカウンター = {state['counter']}")
    new_counter = state["counter"] + 1
    if state["messages"]:
        print(f"counter_node: 最新メッセージ = '{state['messages'][-1].content}'")
    return {"counter": new_counter, "messages": [AIMessage(content=f"カウンターが {new_counter} になりました")]}

def check_finish(state: CheckpointState):
    if state["counter"] >= 3:
        print("check_finish: カウンターが3以上なので終了します。")
        return "__end__"
    else:
        print(f"check_finish: カウンターは {state['counter']} なので続行します。")
        return "continue_counting"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(CheckpointState)
workflow.add_node("counter", counter_node)
workflow.set_entry_point("counter")
workflow.add_conditional_edges(
    "counter",
    check_finish,
    {
        "continue_counting": "counter",
        "__end__": END
    }
)

# --- チェックポインターの設定 ---
memory_saver = ____() 

app = workflow.compile(checkpointer=____) 

# --- グラフの実行と状態の永続化・再開 ---
thread_id = str(uuid.uuid4())
config_1 = {"configurable": {"thread_id": ____}} 

print("\n--- 1回目の実行 (カウンターを1に) ---")
inputs_1 = {"messages": [HumanMessage(content="最初の呼び出し")], "counter": 0}
result_1 = app.invoke(inputs_1, config=config_1)
print(f"1回目 結果: {result_1}")

print("\n--- 2回目の実行 (状態を再開し、カウンターを2に) ---")
inputs_2 = {"messages": [HumanMessage(content="2回目の呼び出し")]}
result_2 = app.invoke(inputs_2, config=____) 
print(f"2回目 結果: {result_2}")

print("\n--- 3回目の実行 (状態を再開し、カウンターを3にして終了) ---")
inputs_3 = {"messages": [HumanMessage(content="3回目の呼び出し")]}
result_3 = app.invoke(inputs_3, config=config_1)
print(f"3回目 結果: {result_3}")

print("\n--- 新しいスレッドでの実行 (カウンターは1から開始) ---")
thread_id_new = str(uuid.uuid4())
config_new = {"configurable": {"thread_id": thread_id_new}}
inputs_new = {"messages": [HumanMessage(content="新しいスレッドの呼び出し")], "counter": 0}
result_new = app.invoke(inputs_new, config=config_new)
print(f"新スレッド 結果: {result_new}")

<details><summary>解答003</summary>

```python
# 解答003
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver # MemorySaverをインポート
import uuid

# --- 状態定義 (State) ---
class CheckpointState(TypedDict):
    messages: Annotated[list, add_messages]
    counter: int

# --- ノード定義 (Nodes) ---
def counter_node(state: CheckpointState):
    print(f"counter_node: 現在のカウンター = {state['counter']}")
    new_counter = state["counter"] + 1
    # ユーザーからの最新メッセージも表示（もしあれば）
    if state["messages"]:
        print(f"counter_node: 最新メッセージ = '{state['messages'][-1].content}'")
    return {"counter": new_counter, "messages": [AIMessage(content=f"カウンターが {new_counter} になりました")]}

def check_finish(state: CheckpointState):
    #カウンターが3以上なら終了、それ以外はcounter_nodeへ
    if state["counter"] >= 3:
        print("check_finish: カウンターが3以上なので終了します。")
        return "__end__" # ENDノードへの特別な名前
    else:
        print(f"check_finish: カウンターは {state['counter']} なので続行します。")
        return "continue_counting"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(CheckpointState)
workflow.add_node("counter", counter_node)

workflow.set_entry_point("counter")

workflow.add_conditional_edges(
    "counter",
    check_finish,
    {
        "continue_counting": "counter",
        "__end__": END
    }
)

# --- チェックポインターの設定 ---
memory_saver = MemorySaver()

# グラフのコンパイル時にチェックポインターを渡す
app = workflow.compile(checkpointer=memory_saver)

# --- グラフの実行と状態の永続化・再開 ---
thread_id = str(uuid.uuid4()) # 各実行シーケンスの一意なID
config_1 = {"configurable": {"thread_id": thread_id}}

print("\n--- 1回目の実行 (カウンターを1に) ---")
inputs_1 = {"messages": [HumanMessage(content="最初の呼び出し")], "counter": 0}
result_1 = app.invoke(inputs_1, config=config_1)
print(f"1回目 結果: {result_1}")
assert result_1['counter'] == 1, f"期待値1, 実際値{result_1['counter']}"

print("\n--- 2回目の実行 (状態を再開し、カウンターを2に) ---")
# 2回目の入力では counter を指定しない -> 保存された状態から再開されるはず
inputs_2 = {"messages": [HumanMessage(content="2回目の呼び出し")]}
result_2 = app.invoke(inputs_2, config=config_1) # 同じthread_idを使用
print(f"2回目 結果: {result_2}")
assert result_2['counter'] == 2, f"期待値2, 実際値{result_2['counter']}"

print("\n--- 3回目の実行 (状態を再開し、カウンターを3にして終了) ---")
inputs_3 = {"messages": [HumanMessage(content="3回目の呼び出し")]}
result_3 = app.invoke(inputs_3, config=config_1)
print(f"3回目 結果: {result_3}")
assert result_3['counter'] == 3, f"期待値3, 実際値{result_3['counter']}"
assert result_3['messages'][-1].content == "カウンターが 3 になりました", "最終メッセージが期待通りではありません"

print("\n--- 新しいスレッドでの実行 (カウンターは1から開始) ---")
thread_id_new = str(uuid.uuid4())
config_new = {"configurable": {"thread_id": thread_id_new}}
inputs_new = {"messages": [HumanMessage(content="新しいスレッドの呼び出し")], "counter": 0}
result_new = app.invoke(inputs_new, config=config_new)
print(f"新スレッド 結果: {result_new}")
assert result_new['counter'] == 1, f"新スレッド期待値1, 実際値{result_new['counter']}"
```
</details>

<details><summary>解説003</summary>

#### この問題のポイント
*   **学習内容:**
    1.  `MemorySaver` の基本的な使い方。
    2.  グラフコンパイル時の `checkpointer` 引数の指定方法。
    3.  `app.invoke` や `app.stream` 時に `config={"configurable": {"thread_id": "some_unique_id"}}` を渡すことで、特定の実行シーケンス（スレッド）の状態を管理する方法。
    4.  同じ `thread_id` を使って `invoke` を呼び出すと、前回の状態から再開されること。
    5.  新しい `thread_id` を使うと、初期状態から開始されること。
*   **コード解説:**
    *   `CheckpointState` に `counter` を追加し、ノードが実行されるたびにインクリメントされるようにしました。
    *   `counter_node` はカウンターを1増やし、現在のカウンター値をメッセージとして返します。
    *   `check_finish` は条件分岐ノードで、カウンターが3以上になるとグラフを終了 (`END`) し、そうでなければ `counter_node` に戻って処理を続行 (`continue_counting`) します。
    *   `memory_saver = MemorySaver()` でインメモリのチェックポインターを作成します。
    *   `app = workflow.compile(checkpointer=memory_saver)` で、コンパイルされたアプリケーションにチェックポインターを接続します。
    *   `thread_id = str(uuid.uuid4())` で、この一連の実行のためのユニークなIDを生成します。
    *   `config_1 = {"configurable": {"thread_id": thread_id}}` の形式でコンフィグを作成し、`app.invoke` 時に渡します。これにより、LangGraphはこのスレッドIDに関連付けられた状態を `memory_saver` から探し、存在すればそこから再開します。
    *   最初の `app.invoke(inputs_1, config=config_1)` では、`inputs_1` で `counter: 0` を渡しているため、初期状態から開始されます。`counter_node` が実行され、カウンターは1になります。この状態が `thread_id` に関連付けられて保存されます。
    *   2回目の `app.invoke(inputs_2, config=config_1)` では、同じ `config_1` (つまり同じ `thread_id`) を使用します。入力 `inputs_2` には `counter` を含めていません。LangGraphは `memory_saver` から `thread_id` の状態（カウンターが1の状態）をロードし、そこからグラフの実行を再開します。そのため、`counter_node` が再度実行され、カウンターは2になります。
    *   3回目の呼び出しも同様にカウンターが3になり、`check_finish` で `END` に遷移します。
    *   最後に、新しい `thread_id_new` を使って実行すると、`memory_saver` にはそのIDの状態が存在しないため、`inputs_new` で指定された初期状態（`counter: 0`）からグラフが開始され、カウンターは1になります。これにより、異なる実行コンテキストが正しく分離されていることが確認できます。
---
</details>

### ■ 問題004: `SqliteSaver` を使った永続化

`MemorySaver` は手軽ですが、プログラムが終了すると状態は失われます。より永続的な保存のためには、データベースバックエンドのチェックポインターを使用します。`SqliteSaver` は、SQLiteデータベースファイルにグラフの状態を保存します。
この問題では、`SqliteSaver` を設定し、グラフの実行状態がSQLiteファイルに保存され、異なるプログラムの実行や長期間の後でも再開できることを（概念的に）示します。実際にファイルが作成されることを確認します。

In [ ]:
# 解答欄004
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from langgraph.checkpoint.sqlite import SqliteSaver # SqliteSaverをインポート
import sqlite3
import uuid
import os

class CheckpointState(TypedDict):
    messages: Annotated[list, add_messages]
    counter: int

def counter_node_sqlite(state: CheckpointState):
    print(f"counter_node_sqlite: 現在のカウンター = {state['counter']}")
    new_counter = state["counter"] + 1
    if state["messages"]:
        print(f"counter_node_sqlite: 最新メッセージ = '{state['messages'][-1].content}'")
    return {"counter": new_counter, "messages": [AIMessage(content=f"カウンターが {new_counter} になりました (sqlite)")]}

def check_finish_sqlite(state: CheckpointState):
    if state["counter"] >= 2:
        print("check_finish_sqlite: カウンターが2以上なので終了します。")
        return "__end__"
    else:
        print(f"check_finish_sqlite: カウンターは {state['counter']} なので続行します。")
        return "continue_counting"

workflow_sqlite = StateGraph(CheckpointState)
workflow_sqlite.add_node("counter_sqlite_node", counter_node_sqlite)
workflow_sqlite.set_entry_point("counter_sqlite_node")
workflow_sqlite.add_conditional_edges(
    "counter_sqlite_node",
    check_finish_sqlite,
    {
        "continue_counting": "counter_sqlite_node",
        "__end__": END
    }
)

db_file = "langgraph_checkpoints.sqlite"
if os.path.exists(db_file):
    os.remove(db_file)

sqlite_conn = ____.____(db_file, check_same_thread=False) 
sqlite_saver = ____(conn=____) 

app_sqlite = workflow_sqlite.compile(checkpointer=____) 

thread_id_sqlite = str(uuid.uuid4())
config_sqlite_1 = {"configurable": {"thread_id": ____}} 

print("\n--- SQLite 1回目の実行 (カウンターを1に) ---")
inputs_sqlite_1 = {"messages": [HumanMessage(content="SQLite最初の呼び出し")], "counter": 0}
result_sqlite_1 = app_sqlite.invoke(inputs_sqlite_1, config=config_sqlite_1)
print(f"SQLite 1回目 結果: {result_sqlite_1}")

print("\n--- DBコネクションを一度閉じて再オープン ---")
sqlite_conn.close()
sqlite_conn_reopened = sqlite3.connect(db_file, check_same_thread=False)
sqlite_saver_reopened = SqliteSaver(conn=sqlite_conn_reopened)
app_sqlite_reopened = workflow_sqlite.compile(checkpointer=sqlite_saver_reopened)

print("\n--- SQLite 2回目の実行 (状態を再開し、カウンターを2にして終了) ---")
inputs_sqlite_2 = {"messages": [HumanMessage(content="SQLite 2回目の呼び出し")]}
result_sqlite_2 = app_sqlite_reopened.invoke(inputs_sqlite_2, config=config_sqlite_1)
print(f"SQLite 2回目 結果: {result_sqlite_2}")

sqlite_conn_reopened.close()

<details><summary>解答004</summary>

```python
# 解答004
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from langgraph.checkpoint.sqlite import SqliteSaver # SqliteSaverをインポート
import sqlite3
import uuid
import os

# --- 状態定義 (State) は問題003と同じものを使用 ---
class CheckpointState(TypedDict):
    messages: Annotated[list, add_messages]
    counter: int

# --- ノード定義 (Nodes) は問題003と同じものを使用 ---
def counter_node_sqlite(state: CheckpointState): # 関数名を変更して区別
    print(f"counter_node_sqlite: 現在のカウンター = {state['counter']}")
    new_counter = state["counter"] + 1
    if state["messages"]:
        print(f"counter_node_sqlite: 最新メッセージ = '{state['messages'][-1].content}'")
    return {"counter": new_counter, "messages": [AIMessage(content=f"カウンターが {new_counter} になりました (sqlite)")]}

def check_finish_sqlite(state: CheckpointState): # 関数名を変更して区別
    if state["counter"] >= 2: # この問題では2回で終了させてシンプルに
        print("check_finish_sqlite: カウンターが2以上なので終了します。")
        return "__end__"
    else:
        print(f"check_finish_sqlite: カウンターは {state['counter']} なので続行します。")
        return "continue_counting"

# --- グラフ構築 (Graph) は問題003と同様のロジック ---
workflow_sqlite = StateGraph(CheckpointState)
workflow_sqlite.add_node("counter_sqlite_node", counter_node_sqlite) # ノード名を変更
workflow_sqlite.set_entry_point("counter_sqlite_node")
workflow_sqlite.add_conditional_edges(
    "counter_sqlite_node",
    check_finish_sqlite,
    {
        "continue_counting": "counter_sqlite_node",
        "__end__": END
    }
)

# --- SQLiteチェックポインターの設定 ---
db_file = "langgraph_checkpoints.sqlite"
# 既存のDBファイルがあれば削除してクリーンな状態で開始
if os.path.exists(db_file):
    os.remove(db_file)
    print(f"既存のDBファイル '{db_file}' を削除しました。")

sqlite_conn = sqlite3.connect(db_file, check_same_thread=False) # sqlite3.connect, check_same_thread=False を追加
sqlite_saver = SqliteSaver(conn=sqlite_conn)

# グラフのコンパイル時にチェックポインターを渡す
app_sqlite = workflow_sqlite.compile(checkpointer=sqlite_saver)

# --- グラフの実行と状態の永続化・再開 ---
thread_id_sqlite = str(uuid.uuid4())
config_sqlite_1 = {"configurable": {"thread_id": thread_id_sqlite}}

print("\n--- SQLite 1回目の実行 (カウンターを1に) ---")
inputs_sqlite_1 = {"messages": [HumanMessage(content="SQLite最初の呼び出し")], "counter": 0}
result_sqlite_1 = app_sqlite.invoke(inputs_sqlite_1, config=config_sqlite_1)
print(f"SQLite 1回目 結果: {result_sqlite_1}")
assert result_sqlite_1['counter'] == 1

# ここで一度コネクションを閉じて、再度開くことで、永続化をより明確にテスト
print("\n--- DBコネクションを一度閉じて再オープン ---")
sqlite_conn.close()
sqlite_conn_reopened = sqlite3.connect(db_file, check_same_thread=False) # check_same_thread=False を追加
sqlite_saver_reopened = SqliteSaver(conn=sqlite_conn_reopened)
app_sqlite_reopened = workflow_sqlite.compile(checkpointer=sqlite_saver_reopened)

print("\n--- SQLite 2回目の実行 (状態を再開し、カウンターを2にして終了) ---")
inputs_sqlite_2 = {"messages": [HumanMessage(content="SQLite 2回目の呼び出し")]}
# configは同じ thread_id を持つものを使用
result_sqlite_2 = app_sqlite_reopened.invoke(inputs_sqlite_2, config=config_sqlite_1)
print(f"SQLite 2回目 結果: {result_sqlite_2}")
assert result_sqlite_2['counter'] == 2

print(f"\nSQLiteデータベースファイル '{db_file}' が作成/更新されました。")
assert os.path.exists(db_file), f"DBファイル '{db_file}' が見つかりません。"

# クリーンアップ（通常は不要だが、テスト実行のために）
sqlite_conn_reopened.close()
# if os.path.exists(db_file):
#     os.remove(db_file)
#     print(f"DBファイル '{db_file}' をクリーンアップしました。")
```
</details>

<details><summary>解説004</summary>

#### この問題のポイント
*   **学習内容:**
    1.  `SqliteSaver` の基本的な使い方と、SQLiteデータベースへの接続方法。
    2.  状態がSQLiteファイルに永続化され、異なるセッションやアプリケーションの再起動後も（同じDBファイルとスレッドIDを使えば）状態を復元できることの理解。
    3.  実際にSQLiteデータベースファイル (`.sqlite`) が作業ディレクトリに作成されることの確認。
    4.  `sqlite3.connect` 時の `check_same_thread=False` の指定について（LangGraphが内部で異なるスレッドからDBアクセスする可能性があるため）。
*   **コード解説:**
    *   グラフの定義やロジックは問題003（`MemorySaver`）とほぼ同じですが、カウンターの終了条件を2に変更してシンプルにし、関数名やノード名に `_sqlite` を追加して区別しやすくしています。
    *   `db_file = "langgraph_checkpoints.sqlite"` でデータベースファイル名を定義します。
    *   `if os.path.exists(db_file): os.remove(db_file)`: この部分は、演習を繰り返し実行する際に、前回の状態が残らないようにするためのおまじないです。
    *   `sqlite_conn = sqlite3.connect(db_file, check_same_thread=False)` でSQLiteデータベースへの接続を確立します。`check_same_thread=False` は、LangGraphが内部で非同期処理や異なるスレッドからデータベースにアクセスする場合があるため、推奨される設定です。これがないと、スレッド関連のエラーが発生することがあります。
    *   `sqlite_saver = SqliteSaver(conn=sqlite_conn)` で、確立した接続を使って`SqliteSaver`のインスタンスを作成します。
    *   最初の `invoke` 後、`sqlite_conn.close()` で一度データベース接続を閉じ、再度接続を開いています。これは、状態がメモリだけでなく、実際にDBファイルに書き込まれ、そこから読み出せることをより明確に示すための手順です。
    *   2回目の `invoke` では、同じ `thread_id_sqlite` を使って、再オープンされたDB接続を持つアプリケーション (`app_sqlite_reopened`) を呼び出します。
    *   最後に `os.path.exists(db_file)` で、実際にデータベースファイルが作成されていることを確認します。
---
</details>

### ■ 問題005: カスタムエージェントの状態に独自の情報を追加・更新する

LangGraphの `AgentState` は通常、`messages` フィールドを含みますが、エージェントの動作をより細かく制御したり、追加情報を追跡したりするために、状態をカスタマイズしたい場合があります。例えば、エージェントが特定のツールを何回呼び出したか、特定の状態を何回経験したかなどを記録できます。
この問題では、標準のメッセージリストに加えて、カスタムフィールド（例: `tool_call_count`）を状態に定義し、エージェントのノードがそのカスタムフィールドを更新する方法を学びます。

In [ ]:
# 解答欄005
from typing import TypedDict, Annotated, Sequence, Literal, Optional
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI 
import operator

@tool
def get_weather(city: str):
    """指定された都市の現在の天気を取得します。"""
    if city == "東京":
        return "東京の天気は晴れです。"
    elif city == "大阪":
        return "大阪の天気は雨です。"
    else:
        return f"{city}の天気は不明です。"

tools = [get_weather]
llm_with_tools = llm.bind_tools(tools)

class CustomAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    tool_call_count: Annotated[int, ____] 
    last_tool_name: Optional[str] 

def agent_node(state: CustomAgentState):
    print(f"エージェントノード実行: 現在のツール呼び出し回数 = {state['tool_call_count']}")
    response = ____.____(state["messages"]) 
    return {"messages": [response]}

def tool_executor_node(state: CustomAgentState):
    print("ツール実行ノード実行")
    ai_message = state["messages"][-1]
    tool_calls = ai_message.tool_calls
    tool_outputs = []
    current_tool_call_count = state.get("tool_call_count", 0)
    last_tool_name = state.get("last_tool_name") # 前回のツール名を取得

    if tool_calls:
        for tc in tool_calls:
            selected_tool = {tool.name: tool for tool in tools}[tc["name"]]
            tool_output = selected_tool.invoke(tc["args"])
            tool_outputs.append(ToolMessage(content=str(tool_output), tool_call_id=tc["id"]))
            # current_tool_call_count は Annotated により自動加算されるので、ここでは差分(1)を返す
            last_tool_name = tc["name"]
            print(f"  ツール '{tc['name']}' を実行しました。")
    
    updates = {"messages": tool_outputs}
    if tool_calls: # ツールが実際に呼び出された場合のみカウントを増やす
        updates[____] = 1 # operator.add が現在の値にこれを加算する
    if last_tool_name:
        updates[____] = last_tool_name
    return updates

def should_continue(state: CustomAgentState) -> Literal["tools_node", "__end__"]:
    if isinstance(state["messages"][-1], AIMessage) and state["messages"][-1].tool_calls:
        return "tools_node"
    return "__end__"

custom_agent_workflow = ____(CustomAgentState) 

custom_agent_workflow.add_node("agent", agent_node)
custom_agent_workflow.add_node("tools_node", tool_executor_node)
custom_agent_workflow.set_entry_point("agent")
custom_agent_workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools_node": "tools_node",
        "__end__": END
    }
)
custom_agent_workflow.add_edge("tools_node", "agent")

app_custom_agent = custom_agent_workflow.____() # compile

initial_state = {
    "messages": [HumanMessage(content="東京の天気を教えてください。その後、大阪の天気もお願いします。") ],
    "tool_call_count": ____, 
    "last_tool_name": None
}
print(f"\n--- カスタムエージェント実行 (初期状態: {initial_state}) ---")
final_state = app_custom_agent.invoke(initial_state)

print("\n--- 最終状態 ---")
for message in final_state["messages"]:
    print(f"  {message.type}: {message.content}")
    if isinstance(message, AIMessage) and message.tool_calls:
        print(f"    ツール呼び出し: {message.tool_calls}")
print(f"最終ツール呼び出し回数: {final_state['tool_call_count']}")
print(f"最後に呼び出されたツール: {final_state['last_tool_name']}")

<details><summary>解答005</summary>

```python
# 解答005
from typing import TypedDict, Annotated, Sequence, Literal, Optional
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI # この問題ではLLMを直接使うためインポート
import operator

# --- ツールの定義 ---
@tool
def get_weather(city: str):
    """指定された都市の現在の天気を取得します。"""
    if city == "東京":
        return "東京の天気は晴れです。"
    elif city == "大阪":
        return "大阪の天気は雨です。"
    else:
        return f"{city}の天気は不明です。"

tools = [get_weather]
# OpenAIファンクションコーリングを利用する場合、LLMにツールをバインドする
# ここではノートブック冒頭の `llm` を使う想定だが、この問題専用のLLMインスタンスを作成しても良い
llm_with_tools = llm.bind_tools(tools)

# --- カスタム状態定義 (State) ---
class CustomAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    tool_call_count: Annotated[int, operator.add] # ツール呼び出し回数をカウントするカスタムフィールド
    last_tool_name: Optional[str] # 最後に呼び出されたツール名（デモ用）

# --- エージェントノード定義 ---
def agent_node(state: CustomAgentState):
    print(f"エージェントノード実行: 現在のツール呼び出し回数 = {state['tool_call_count']}")
    # LLMに必要な情報を渡して応答を生成
    # state['messages'] にはユーザー入力や過去のAI応答、ツール結果が含まれる
    response = llm_with_tools.invoke(state["messages"]) 
    return {"messages": [response]}

# --- ツール実行ノード定義 ---
def tool_executor_node(state: CustomAgentState):
    print("ツール実行ノード実行")
    ai_message = state["messages"][-1] # 直前のAIメッセージ（ツール呼び出しを含むはず）
    tool_calls = ai_message.tool_calls
    tool_outputs = []
    # tool_call_count は Annotated の operator.add で自動加算されるので、ここでは更新差分を返す
    # last_tool_name は単純な上書き
    num_tool_calls_in_this_step = 0
    current_last_tool_name = None

    if tool_calls:
        for tc in tool_calls:
            selected_tool = {tool.name: tool for tool in tools}[tc["name"]]
            tool_output = selected_tool.invoke(tc["args"])
            tool_outputs.append(ToolMessage(content=str(tool_output), tool_call_id=tc["id"]))
            num_tool_calls_in_this_step += 1
            current_last_tool_name = tc["name"]
            print(f"  ツール '{tc['name']}' を実行しました。 (このステップでの呼び出し: {num_tool_calls_in_this_step})")
    
    updates = {"messages": tool_outputs}
    if num_tool_calls_in_this_step > 0:
        updates["tool_call_count"] = num_tool_calls_in_this_step 
    if current_last_tool_name:
        updates["last_tool_name"] = current_last_tool_name
    return updates

# --- ルーティングロジック ---
def should_continue(state: CustomAgentState) -> Literal["tools_node", "__end__"]:
    if isinstance(state["messages"][-1], AIMessage) and state["messages"][-1].tool_calls:
        print("ルーティング: ツール呼び出しあり -> tool_executor_node へ")
        return "tools_node"
    print("ルーティング: ツール呼び出しなし -> 終了")
    return "__end__"

# --- グラフ構築 ---
custom_agent_workflow = StateGraph(CustomAgentState) 

custom_agent_workflow.add_node("agent", agent_node)
custom_agent_workflow.add_node("tools_node", tool_executor_node)

custom_agent_workflow.set_entry_point("agent")

custom_agent_workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools_node": "tools_node",
        "__end__": END
    }
)
custom_agent_workflow.add_edge("tools_node", "agent") # ツール実行後、再度エージェントノードへ

app_custom_agent = custom_agent_workflow.compile()

# --- グラフ実行 ---
initial_state = {
    "messages": [HumanMessage(content="東京の天気を教えてください。その後、大阪の天気もお願いします。") ],
    "tool_call_count": 0, 
    "last_tool_name": None
}
print(f"\n--- カスタムエージェント実行 (初期状態: {initial_state}) ---")
final_state = app_custom_agent.invoke(initial_state)

print("\n--- 最終状態 ---")
for message in final_state["messages"]:
    print(f"  {message.type}: {message.content}")
    if isinstance(message, AIMessage) and message.tool_calls:
        print(f"    ツール呼び出し: {message.tool_calls}")
print(f"最終ツール呼び出し回数: {final_state['tool_call_count']}")
print(f"最後に呼び出されたツール: {final_state['last_tool_name']}")

assert final_state['tool_call_count'] >= 1, "ツール呼び出し回数がカウントされていません"
```
</details>

<details><summary>解説005</summary>

#### この問題のポイント
*   **学習内容:**
    1.  `TypedDict` を使って、グラフの状態に独自のフィールド（例: `tool_call_count`, `last_tool_name`）を追加する方法。
    2.  `Annotated[int, operator.add]` のようにアノテーションとリデューサー関数 (この場合は `operator.add`) を組み合わせることで、状態の特定フィールドがどのように更新されるかを定義する方法。これにより、ノードが新しい値を返すだけで、グラフが自動的に値を加算してくれます。
    3.  ノード関数内でこれらのカスタム状態フィールドを読み書きする方法。
    4.  ツール呼び出しを含む標準的なエージェントループ（エージェントノード -> ツール実行ノード -> エージェントノード）の中で、カスタム状態がどのように維持・更新されるか。
*   **コード解説:**
    *   `CustomAgentState`:
        *   `messages: Annotated[Sequence[BaseMessage], add_messages]` は標準的なメッセージ履歴です。
        *   `tool_call_count: Annotated[int, operator.add]` がカスタムフィールドです。`operator.add` を指定することで、このフィールドに新しい値が返されるたびに、既存の値に加算されます。初期値は `invoke` 時に指定します（この例では0）。このリデューサーのおかげで、`tool_executor_node` はそのステップで実行されたツール呼び出しの数（例えば1）を返すだけで、LangGraphが自動的に総カウントに加算します。
        *   `last_tool_name: Optional[str]` もカスタムフィールドで、最後に呼び出されたツール名を保持します。こちらは単純な上書きで更新されます（リデューサーなし）。
    *   `agent_node`:
        *   LLM（`llm_with_tools`）を呼び出し、応答（AIメッセージ、ツール呼び出しを含む可能性あり）を返します。このノード自体は `tool_call_count` を直接変更しません。
    *   `tool_executor_node`:
        *   AIメッセージからツール呼び出し情報を取得し、対応するツールを実行します。
        *   `num_tool_calls_in_this_step` で、このノードの1回の実行中に行われたツール呼び出しの数をカウントします。
        *   返す辞書に `{"tool_call_count": num_tool_calls_in_this_step}` を含めます。`operator.add` のアノテーションにより、グラフはこの値を既存の `tool_call_count` に「加算」します。
        *   `last_tool_name` も更新します。
    *   `should_continue`:
        *   エージェントの応答にツール呼び出しが含まれていれば `tools_node` へ、そうでなければグラフを終了します。
    *   グラフの実行:
        *   `initial_state` で `tool_call_count: 0` と `last_tool_name: None` を初期値として設定します。
        *   エージェントはまず「東京の天気」を尋ねるツール呼び出しを行い、`tool_executor_node` で `tool_call_count` が1に（0+1）、`last_tool_name` が `get_weather` になります。
        *   次にエージェントは「大阪の天気」を尋ねるツール呼び出しを行い、`tool_executor_node` で `tool_call_count` が2に（1+1）なります。
        *   最終的に、`final_state` でこれらのカスタムフィールドが正しく更新されていることを確認します。
---
</details>

### ■ 問題006: エージェントスウォームの概念: スーパーバイザーとワーカーエージェント

エージェントスウォームは、複数の専門エージェントが協調して複雑なタスクを解決するアーキテクチャです。多くの場合、タスクを適切なワーカーエージェントに振り分ける「スーパーバイザー」エージェントが存在します。
この問題では、エージェントスウォームの基本的な概念を理解するために、非常にシンプルなスーパーバイザーと2つのワーカーエージェント（実際には単純なノード）からなるグラフを作成します。スーパーバイザーは、入力メッセージの内容に基づいて、タスクを「一般的な質問応答ワーカー」または「天気情報ワーカー」のいずれかにルーティングします。

In [ ]:
# 解答欄006
from typing import TypedDict, Annotated, Literal, Optional, List
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class SwarmState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    next_node: Optional[str] 
    task_description: str      
    result: Optional[str]      

# --- ワーカーノード定義 ---
def general_qa_worker_node(state: SwarmState):
    print(f"汎用QAワーカー実行: タスク「{state['task_description']}」")
    response = f"汎用QAワーカーがタスク「{state['task_description']}」を処理しました。"
    return {"result": response, "messages": [AIMessage(content=response)]}

def weather_info_worker_node(state: SwarmState):
    print(f"天気情報ワーカー実行: タスク「{state['task_description']}」")
    if "天気" in state['task_description']:
        response = f"天気情報ワーカー: {state['task_description']}の天気は快晴です。"
    else:
        response = f"天気情報ワーカー: 天気に関するタスクではありません。「{state['task_description']}」"
    return {"result": response, "messages": [AIMessage(content=response)]}

# --- スーパーバイザーノード定義 (ルーティングロジック) ---
def supervisor_node(state: SwarmState) -> dict[str, str]:
    print("スーパーバイザー実行")
    task_desc = state["task_description"]
    print(f"  タスク内容: 「{task_desc}」")
    
    if "天気" in task_desc.lower() or "weather" in task_desc.lower():
        print("  ルーティング先: 天気情報ワーカー")
        return { "next_node": ____ } 
    else:
        print("  ルーティング先: 汎用QAワーカー")
        return { "next_node": ____ } 

# --- グラフ構築 ---
swarm_workflow = ____(SwarmState) 

swarm_workflow.add_node("supervisor", supervisor_node)
swarm_workflow.add_node("general_qa_worker", general_qa_worker_node)
swarm_workflow.add_node("weather_worker", weather_info_worker_node)

swarm_workflow.set_entry_point("supervisor")

swarm_workflow.add_conditional_edges(
    "supervisor",
    lambda state: state[____], 
    {
        "general_qa_worker": "general_qa_worker",
        "weather_worker": "weather_worker",
    }
)

swarm_workflow.add_edge("general_qa_worker", ____) 
swarm_workflow.add_edge("weather_worker", ____) 

app_swarm = swarm_workflow.compile()

# --- グラフ実行テスト ---
print("\n--- スウォーム実行テスト1 (天気タスク) ---")
initial_input_weather = {
    "messages": [HumanMessage(content="今日の東京の天気を教えて")],
    "task_description": "今日の東京の天気を教えて", 
}
final_state_weather = app_swarm.invoke(initial_input_weather)
print(f"  最終結果 (天気): {final_state_weather.get('result')}")

print("\n--- スウォーム実行テスト2 (一般タスク) ---")
initial_input_qa = {
    "messages": [HumanMessage(content="LangGraphとは何ですか？")],
    "task_description": "LangGraphとは何ですか？",
}
final_state_qa = app_swarm.invoke(initial_input_qa)
print(f"  最終結果 (QA): {final_state_qa.get('result')}")

<details><summary>解答006</summary>

```python
# 解答006
from typing import TypedDict, Annotated, Literal, Optional, List
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class SwarmState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    next_node: Optional[str] # 次に実行するノード名を保持
    task_description: str      # 現在のタスク記述
    result: Optional[str]      # ワーカーからの結果

# --- ワーカーノード定義 ---
def general_qa_worker_node(state: SwarmState):
    print(f"汎用QAワーカー実行: タスク「{state['task_description']}」")
    # 実際にはLLM呼び出しなどを行う
    response = f"汎用QAワーカーがタスク「{state['task_description']}」を処理しました。"
    return {"result": response, "messages": [AIMessage(content=response)]}

def weather_info_worker_node(state: SwarmState):
    print(f"天気情報ワーカー実行: タスク「{state['task_description']}」")
    # 実際には天気ツール呼び出しなどを行う
    if "天気" in state['task_description']:
        response = f"天気情報ワーカー: {state['task_description']}の天気は快晴です。"
    else:
        response = f"天気情報ワーカー: 天気に関するタスクではありません。「{state['task_description']}」"
    return {"result": response, "messages": [AIMessage(content=response)]}

# --- スーパーバイザーノード定義 (ルーティングロジック) ---
def supervisor_node(state: SwarmState) -> dict[str, str]:
    print("スーパーバイザー実行")
    # messagesから最新のユーザー入力を取得
    # この例では、簡略化のため initial_input で task_description を直接設定
    task_desc = state["task_description"]
    print(f"  タスク内容: 「{task_desc}」")
    
    if "天気" in task_desc.lower() or "weather" in task_desc.lower():
        print("  ルーティング先: 天気情報ワーカー")
        return { "next_node": "weather_worker" }
    else:
        print("  ルーティング先: 汎用QAワーカー")
        return { "next_node": "general_qa_worker" }

# --- グラフ構築 ---
swarm_workflow = StateGraph(SwarmState)

swarm_workflow.add_node("supervisor", supervisor_node)
swarm_workflow.add_node("general_qa_worker", general_qa_worker_node)
swarm_workflow.add_node("weather_worker", weather_info_worker_node)

swarm_workflow.set_entry_point("supervisor")

# スーパーバイザーから各ワーカーへの条件付きエッジ
swarm_workflow.add_conditional_edges(
    "supervisor",
    lambda state: state["next_node"], # next_node の値に基づいて遷移
    {
        "general_qa_worker": "general_qa_worker",
        "weather_worker": "weather_worker",
        # 想定外の next_node の値の場合のフォールバック (オプション)
        # "__end__": END 
    }
)

# 各ワーカーの処理が終わったらグラフを終了
swarm_workflow.add_edge("general_qa_worker", END)
swarm_workflow.add_edge("weather_worker", END)

app_swarm = swarm_workflow.compile()

# --- グラフ実行テスト ---
print("\n--- スウォーム実行テスト1 (天気タスク) ---")
initial_input_weather = {
    "messages": [HumanMessage(content="今日の東京の天気を教えて")],
    "task_description": "今日の東京の天気を教えて", # スーパーバイザーがこの情報を使う
}
final_state_weather = app_swarm.invoke(initial_input_weather)
print(f"  最終結果 (天気): {final_state_weather.get('result')}")
assert "快晴" in final_state_weather.get('result', ""), "天気の応答が期待通りではありません。"

print("\n--- スウォーム実行テスト2 (一般タスク) ---")
initial_input_qa = {
    "messages": [HumanMessage(content="LangGraphとは何ですか？")],
    "task_description": "LangGraphとは何ですか？",
}
final_state_qa = app_swarm.invoke(initial_input_qa)
print(f"  最終結果 (QA): {final_state_qa.get('result')}")
assert "汎用QAワーカー" in final_state_qa.get('result', ""), "QA応答が期待通りではありません。"
```
</details>

<details><summary>解説006</summary>

#### この問題のポイント
*   **学習内容:**
    1.  エージェントスウォームにおけるスーパーバイザーの役割（タスクの振り分け）の概念的な理解。
    2.  状態 (`SwarmState`) にルーティング情報 (`next_node`) を持たせ、スーパーバイザーノードがこの情報を更新する方法。
    3.  `add_conditional_edges` を使用して、スーパーバイザーノードの決定に基づいて異なるワーカーノードに処理を分岐させる方法。
    4.  ワーカーノードがそれぞれの専門タスクを処理し、結果を状態に返す基本的な流れ。
*   **コード解説:**
    *   `SwarmState`:
        *   `messages`: 会話履歴。
        *   `next_node`: スーパーバイザーが次にどのワーカーを実行すべきかを格納します。
        *   `task_description`: 現在処理中のタスクの記述。実際のスーパーバイザーは `messages` からこれを抽出・生成しますが、この例では簡略化のため初期入力で与えます。
        *   `result`: ワーカーノードからの処理結果を格納します。
    *   `general_qa_worker_node`, `weather_info_worker_node`:
        *   それぞれの専門タスクを模倣する単純な関数です。実際のアプリケーションでは、これらのノードは自身がLLMエージェントであったり、特定のツールセットを持っていたりします。
        *   処理結果を `result` フィールドに、そしてユーザー向けの応答を `messages` に追加して返します。
    *   `supervisor_node`:
        *   `task_description` を見て、タスクの内容に応じて `next_node` の値を `"weather_worker"` または `"general_qa_worker"` に設定します。
        *   このノードが返すのは、状態の更新差分 (`{"next_node": ...}`) です。
    *   グラフ構築:
        *   スーパーバイザーノードと2つのワーカーノードを追加します。
        *   エントリーポイントは `supervisor` です。
        *   `add_conditional_edges`:
            *   `supervisor` ノードの後に実行されます。
            *   2番目の引数 `lambda state: state["next_node"]` は、状態の `next_node` フィールドの値（例: `"weather_worker"`）を返します。
            *   3番目の引数の辞書は、この返された値と実際の遷移先ノード名をマッピングします。例えば、`next_node` が `"weather_worker"` なら `weather_worker` ノードに遷移します。
        *   各ワーカーノードの処理が終わったら `END` に遷移し、グラフを終了します。
    *   実行テスト:
        *   「天気タスク」と「一般タスク」の2つのシナリオでグラフを実行し、スーパーバイザーが正しくルーティングし、適切なワーカーが応答を生成することを確認します。
*   **補足:**
    *   これは非常に単純化されたスウォームの例です。実際の高度なエージェントスウォームでは、スーパーバイザーが複数のワーカーに並列でタスクを依頼したり、ワーカー間で連携したり、結果を集約したりするなど、より複雑なロジックを持ちます。
    *   LangGraphの `Pregel` エンジンは、このような複雑なマルチエージェントのワークフローを構築するのに非常に強力です。
---
</details>

In [ ]:
# 今後の実装に向けた空のコードセル